In [1]:
# Librarys
%run ForeSignalScraper.ipynb function(Login)
%run ForeSignalScraper.ipynb function(CustomizeFoSignals)
%run TradingViewScaper.ipynb function(Price)
%run OpenOrdersMT5.ipynb function(Login_mt5,Expiration_time,Determination_Set_Order)
from datetime import datetime
from collections import OrderedDict
import pandas as pd
import random
import time

In [2]:
# Currencys list and Desired Time to Refresh wbsites
Currencys_list = ['EURUSD','USDCHF','GBPUSD','USDJPY','USDCAD','AUDUSD','EURJPY','NZDUSD','GBPCHF']
desired_time = int(input('Please Enter Your Desired Time Range To load Websiter - input a number ex:20 : '))

# Foresignal login Information
FoLogin_url = 'https://foresignal.com/en/login/index'
Fouser_name = 'xabiz'
FoUser_password = 'tohid0010011'

# Foresignal Scrape Specification
FoWebsite_url = 'https://foresignal.com/en/'
Foclassorid = 'class'
Foclassoridvalue = 'd-flex flex-row flex-wrap signal-row'

# MetaTrader5 login Information
login_id = 3060009
password = '$9Bj#5qL'
server = 'FIBOGroup-MT5 Server'
Login_mt5 (login_id,password,server)

counter=0
final_list = Price(2,Currencys_list)
ForeSignalDriver = Login(FoLogin_url,Fouser_name,FoUser_password)
final_informations = OrderedDict()
main_list = list()
FoFinalList = list()
Reference_list = list()


Please Enter Your Desired Time Range To load Websiter - input a number ex:20 : 60


In [ ]:
while True:

    FoFinalList_Start_len = len(FoFinalList)

    FoFinalList = CustomizeFoSignals(ForeSignalDriver,FoWebsite_url,Foclassorid,Foclassoridvalue)

    FoFinalList_Final_len = len(FoFinalList)

    Tradingview_price_dictionary = OrderedDict()

    delay_time = random.randrange(0,desired_time)

    tradnigview_scrape_counter = 0

    if FoFinalList_Start_len<FoFinalList_Final_len:
        
        main_list_start_len = len (main_list)

        for items in FoFinalList:
            if len(items) > 10 and items not in main_list:
                main_list.append(items)
        
        main_list_final_len = len (main_list)
        
        for item in range (0,9):
            now = datetime.now()
            final_list[0].switch_to.window(final_list[0].window_handles[item])
            Tradingview_price_dictionary[f'{Currencys_list[item]} : {now}'] = final_list[1][item].text
        # print(Tradingview_price_dictionary)

        for item_fo in main_list:
            for k,v in Tradingview_price_dictionary.items():
                if item_fo[0] in k[:6]:
                    item_fo.append(k)
                    item_fo.append(v)
                    final_informations[f'{item_fo[:11]}']=item_fo
        
        Reference_list_start_len = len(Reference_list)

        Reference_list = list(final_informations.items()) 
        
        Reference_list_final_len = len(Reference_list)

        for items in Reference_list[Reference_list_start_len:Reference_list_final_len]:
            
            print(items[1])
            Symbole = items[1][0]
            Entry_Price = float(items[1][6])
            Stoploss_Price = float(items[1][10])
            Takeprofit_Price = float(items[1][8])
            Current_Price = float(items[1][12])
            current_time = items[1][4].replace(':',' ').split()
            expiration = Expiration_time (int(current_time[0]),int(current_time[1]))
                                       
            
            if items[1][5] == 'Buy at' and Entry_Price > Current_Price:
                Type_of_Trade = 'buy stop'
                
            elif items[1][5] == 'Buy at' and Entry_Price < Current_Price:
                Type_of_Trade = 'buy limit'
            
            elif items[1][5] == 'Sell at' and Entry_Price < Current_Price:
                Type_of_Trade = 'sell stop'
            
            elif items[1][5] == 'Sell at' and Entry_Price > Current_Price:
                Type_of_Trade = 'sell limit'
                
            request = Determination_Set_Order(Type_of_Trade, Symbole , 0.05 ,Entry_Price
                                              ,Stoploss_Price,Takeprofit_Price,expiration)   
            result = mt.order_send(request)
            result

    time.sleep(delay_time)